In [1]:
from ipyleaflet import *
import ipywidgets
from ipywidgets.embed import embed_minimal_html
from ipywebrtc import WidgetStream, ImageRecorder

import json
import numpy as np
import matplotlib.pyplot as plt
#import gdal

import rasterio
from rasterio import plot
from rasterio._base import gdal_version
import contextily

import ee
from geemap import *

import osmnx as ox
# from OSMPythonTools.api import OSM_api

In [2]:
# To run once for setting up earth engine
#ee.Authenticate()

In [3]:
ee.Initialize()

# Fetch satellite images

In [4]:
class SatelliteData:
    def __init__(self, name, src, vis, zoom):
        self.name = name
        self.src = src
        self.vis = vis
        self.zoom = zoom
        
    def getData(self):
        return 'Not defined'
        
    def getDataWithDate(self, date_start, date_end):
        return 'Not defined'
        
    def getDataWithArea(self, aoi, date_start, date_end):
        return 'Not defined'

In [5]:
class Sentinel2(SatelliteData):
    def __init__(self):
        self.name = 'Sentinel-2'
        self.src = 'COPERNICUS/S2_SR'
        self.vis = {
            "min": 0, 
            "max": 3000, 
            "bands": ['B4', 'B3', 'B2'],
        }
        self.zoom = 14
        
    def getData(self):
        return (ee.ImageCollection(self.src)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1)))
    
    def getDataWithDate(self, date_start, date_end):
        return (ee.ImageCollection(self.src)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
                .filterDate(date_start, date_end))
    
    def getDataWithArea(self, aoi, date_start, date_end):
        return (ee.ImageCollection(self.src)
                .filterDate(date_start, date_end)
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 1))
                .mosaic().clip(aoi).unmask())
    
class Landsat8(SatelliteData):
    def __init__(self):
        self.name = 'Landsat-8'
        self.src = 'LANDSAT/LC08/C01/T1_SR'
        self.vis = {
            'min': 0, 
            'max': 3000, 
            'bands': ['B4', 'B3', 'B2'],
        }
        self.zoom = 13
        
    def getData(self):
        return (ee.ImageCollection(self.src)
                .filter(ee.Filter.lt('CLOUD_COVER', 20)))
        
    def getDataWithDate(self, date_start, date_end):
        return (ee.ImageCollection(self.src)
                .filterDate(date_start, date_end)
                .filter(ee.Filter.lt('CLOUD_COVER', 20)))
        
    def getDataWithArea(self, aoi, date_start, date_end):
        return (ee.ImageCollection(self.src)
                .filterDate(date_start, date_end)
                .filter(ee.Filter.lt('CLOUD_COVER', 20))
                .mosaic().clip(aoi).unmask())
    
class NightVIIRS(SatelliteData):
    def __init__(self, minRad=1, maxRad=2):
        self.name = 'Night VIIRS'
        self.src = 'NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG'
        self.vis = {
            'min': minRad,
            'max': maxRad, 
            'bands': ['avg_rad'], 
            'palette': ['000000','ffffff'],
        }
        self.zoom = 9
        
    def getData(self):
        return (ee.ImageCollection(self.src))
        
    def getDataWithDate(self, date_start, date_end):
        return (ee.ImageCollection(self.src)
             .filterDate(date_start, date_end))
        
    def getDataWithArea(self, aoi, date_start, date_end):
        return (ee.ImageCollection(self.src)
             .filterDate(date_start, date_end)
             .mosaic().clip(aoi).unmask()
            )

class popGHSL(SatelliteData):
    def __init__(self):
        self.name = 'Population GHSL'
        self.src = 'JRC/GHSL/P2016/POP_GPW_GLOBE_V1'
        self.vis = {
            "min": 0.0,
            "max": 20.0,
            "palette": ['000000', 'ffffff'],
        }
        self.zoom = 11
        
    def getData(self):
        return (ee.ImageCollection(self.src)
                .filterDate('2014-01-01', '2016-01-01')
               ).select('population_count')
    
    def getDataWithDate(self, date_start=None, date_end=None):
        return (ee.ImageCollection(self.src)
                .filterDate('2014-01-01', '2016-01-01')
               ).select('population_count')
        
    def getDataWithArea(self, aoi, date_start, date_end):
        return (ee.ImageCollection(self.src)
                .filterDate('2014-01-01', '2016-01-01')
                .mosaic().clip(aoi).unmask()
               ).select('population_count')

In [6]:
class AOI:
    def __init__(self):
        self.isEmpty = False
    
    xMin = 0
    xMax = 0
    yMin = 0
    yMax = 0

In [7]:
def extractCoordFromFeatureCollection(features, precision):
    aoi = AOI()
    aoi.isEmpty = True
    if features.size().getInfo() != 0:
        coord = features.first().geometry().getInfo()['coordinates']
        coord = np.array(coord).reshape(-1, 2) # Reshap in one array of tuple
        if len(coord) != 0:
            aoi.isEmpty = False
            aoi.yMin = round(min([y for x, y in coord]),precision)
            aoi.xMin = round(min([x for x, y in coord]),precision)
            aoi.yMax = round(max([y for x, y in coord]),precision)
            aoi.xMax = round(max([x for x, y in coord]),precision)
    return aoi

In [8]:
def saveImageFromGeeData(folderName, sat, aoi, aoiEE, date_start, date_end):
    offset = 100
    # lontal : 0.25 square at zoom=14
    refStep = 4096
    step = int(refStep*offset/(2**sat.zoom))
    count = 0
    
    x_min = int(aoi.xMin*offset)
    x_max = int(aoi.xMax*offset)
    y_min = int(aoi.yMin*offset)
    y_max = int(aoi.yMax*offset)
    
    aoiEEsquare = ee.Geometry.Rectangle([aoi.xMin, aoi.yMin, aoi.xMax, aoi.yMax])

    data = sat.getDataWithArea(aoiEE, date_start, date_end).clip(aoiEEsquare)
    url = data.getMapId(sat.vis)["tile_fetcher"].url_format
    
    folderName = 'img/'+folderName
    os.makedirs(folderName,exist_ok=True)
    relativeFolderName = folderName+'/'+sat.name
    fullPath = os.path.join(os.getcwd(),relativeFolderName+'.tif')
    
    # Old attempt
    #geemap.ee_export_image(data, filename=fullPath, scale=14,region=aoiEE,file_per_band=False)
    
    for x in range(x_min, x_max, step):
        for y in range(y_min, y_max, step):
            count += 1
            
            east = x_max/offset if (x+step > x_max) else (x+step)/offset
            west = x/offset
            north = y_max/offset if (y+step > y_max) else (y+step)/offset
            south = y/offset
            
            print(count,'w:',west,'e:',east,'s:',south,'n:',north)
            _, _ = contextily.bounds2raster(west,south,east,north,ll=True,path=relativeFolderName+'_'+str(count)+'.tif',source=url, zoom=sat.zoom)
            print('Done')

In [9]:
def displayArea(m, listSatellite, precision = 3):
        
    def on_save_btn_clicked(e):
        with output:
            aoiEE = ee.FeatureCollection(m.draw_last_feature)
            aoi = extractCoordFromFeatureCollection(aoiEE, precision)
            
            if (datepicker_start.value != None and datepicker_end.value != None 
                and datepicker_start.value < datepicker_end.value 
                and titleField.value != '' 
                and not aoi.isEmpty):
                
                for sat in listSatellite:
                    m.remove_ee_layer(sat.name)

                date_start = datepicker_start.value.strftime('%Y-%m-%d')
                date_end = datepicker_end.value.strftime('%Y-%m-%d')
                
                for sat in listSatellite:
                    print(sat.name)
                    m.addLayer(sat.getDataWithArea(aoiEE, date_start, date_end).clip(aoiEE), sat.vis, sat.name)
                    saveImageFromGeeData(titleField.value, sat, aoi, aoiEE, date_start, date_end)

    def on_add_btn_clicked(e):
        with output:
            aoiEE = ee.FeatureCollection(m.draw_features)
            aoi = extractCoordFromFeatureCollection(aoiEE, precision)
            print(aoi.isEmpty)
            if (datepicker_start.value != None and datepicker_end.value != None):
                for sat in listSatellite:
                    m.remove_ee_layer(sat.name)
                date_start = datepicker_start.value.strftime('%Y-%m-%d')
                date_end = datepicker_end.value.strftime('%Y-%m-%d')
                for sat in listSatellite:
                    if aoi.isEmpty:
                        m.addLayer(sat.getDataWithDate(date_start, date_end).clip(aoiEE), sat.vis, sat.name)
                    else:
                        m.addLayer(sat.getDataWithArea(aoiEE,date_start, date_end).clip(aoiEE), sat.vis, sat.name)

    def show_coordinates(**args):
        latlon = args.get('coordinates')
        if args.get('type') == 'mousemove':
            with output_widget:
                output_widget.clear_output()
                print(latlon)
        
    
    output_widget = widgets.Output()
    output_mouse_coordinates = WidgetControl(widget=output_widget, position='bottomright')
    
    datepicker_start = widgets.DatePicker(description='Start date',disabled=False)
    datepicker_end = widgets.DatePicker(description='End date',disabled=False)
    
    titleField = widgets.Text(
        value='',
        placeholder='Type the project title here',
        description='String:',
        disabled=False
    )
    addBtn = widgets.Button(description="add btn !")
    saveBtn = widgets.Button(description="save btn !")
    output = widgets.Output()
    
    addBtn.on_click(on_add_btn_clicked)
    saveBtn.on_click(on_save_btn_clicked)
    
    m.add_control(output_mouse_coordinates)
    m.on_interaction(show_coordinates)
    
    m.draw_features
    m.draw_last_feature
    
    display(m)
    display(titleField)
    display(datepicker_start)
    display(datepicker_end)
    display(addBtn)
    display(saveBtn)


In [10]:
satData = [NightVIIRS(1,2), popGHSL(), Sentinel2(), Landsat8()]

m = geemap.Map(center=(0, 0), zoom=4)
m.add_control(ScaleControl(position="topright"))
m.add_control(LayersControl())
displayArea(m, satData, 0)


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Toggl…

Text(value='', description='String:', placeholder='Type the project title here')

DatePicker(value=None, description='Start date')

DatePicker(value=None, description='End date')

Button(description='add btn !', style=ButtonStyle())

Button(description='save btn !', style=ButtonStyle())